In [271]:
import pandas as pd

# Wczytanie danych
df = pd.read_csv('C:\\Users\\szymo\\Desktop\\PythonProjects\\PUM_semester_project\\data_refactoring\\cars_data\\bmw\\extracted_bmw_data\\bmw_without_encoded.csv')
df = df.drop_duplicates(subset=['id'], keep='first')
df = df.drop(columns=['id', 'cant_see_my_version'])


In [272]:
combustion_cars = df.loc[df.fuel_type != 'electric', :].copy()
combustion_cars

,is_imported_car,model,year,mileage,engine_capacity,fuel_type,engine_power,gearbox,transmission,bluetooth_interface,...,maker_warranty_valid_until_date,maker_warranty_valid_until_km,convertible_top_type,monthly_payment,remaining_payments,residual_value,down_payment,avg_consumption,show_pir,price_amount
0,0.0,i8,2015,28279,1499.0,hybrid,362.0,automatic,all-wheel-auto,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,299000.00
1,1.0,seria-7,2011,250000,2993.0,diesel,245.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70000.00
2,0.0,3gt,2016,250147,1995.0,diesel,150.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62900.00
3,0.0,seria-5,2018,83000,2993.0,diesel,400.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239900.00
4,1.0,seria-6,2017,133000,2993.0,diesel,313.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139900.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11885,1.0,seria-8,2019,103700,2998.0,petrol,340.0,automatic,rear-wheel,1.0,...,NaN,NaN,textile-top,NaN,NaN,NaN,NaN,NaN,NaN,223900.00
11886,0.0,seria-5,2019,129990,1995.0,diesel,150.0,automatic,rear-wheel,NaN,...,NaN,NaN,NaN,583.0,NaN,NaN,NaN,NaN,NaN,97997.79
11887,0.0,x6,2017,92614,2993.0,diesel,313.0,automatic,all-wheel-auto,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.00
11888,1.0,3gt,2019,199800,1995.0,diesel,143.0,manual,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51900.00


In [273]:

import numpy as np

combustion_cars.is_imported_car = combustion_cars.is_imported_car.astype('Int32')
combustion_cars.is_imported_car.value_counts(dropna=False)
combustion_cars.is_imported_car = combustion_cars.is_imported_car.fillna(np.random.randint(0, 1)) # poniewaz rozklad jest rownomierny zastosuje losowe wartosci


filled = combustion_cars.dropna(subset=['model', 'year', 'generation', 'version'])

most_common_version = (
    filled.groupby(['model', 'year', 'generation', 'version'])
    .size()
    .reset_index(name='count')
)

# Dla każdej kombinacji model-year-generation wybieramy version z największą liczbą
most_common_per_group = (
    most_common_version
    .sort_values(['model', 'year', 'generation', 'count'], ascending=[True, True, True, False])
    .drop_duplicates(subset=['model', 'year', 'generation'])
    .reset_index(drop=True)
)

def fill_version(row):
    if pd.isna(row['version']):
        match = most_common_per_group[
            (most_common_per_group['model'] == row['model']) &
            (most_common_per_group['year'] == row['year']) &
            (most_common_per_group['generation'] == row['generation'])
        ]
        if not match.empty:
            return match.iloc[0]['version']
    return row['version']

combustion_cars['version'] = combustion_cars.apply(fill_version, axis=1)


filled_generation = combustion_cars.dropna(subset=['model', 'year', 'version', 'generation'])


most_common_generation = (
    filled_generation.groupby(['model', 'year', 'version', 'generation'])
    .size()
    .reset_index(name='count')
)

most_common_generation_per_group = (
    most_common_generation
    .sort_values(['model', 'year', 'version', 'count'], ascending=[True, True, True, False])
    .drop_duplicates(subset=['model', 'year', 'version'])
    .reset_index(drop=True)
)

def fill_generation(row):
    if pd.isna(row['generation']):
        match = most_common_generation_per_group[
            (most_common_generation_per_group['model'] == row['model']) &
            (most_common_generation_per_group['year'] == row['year']) &
            (most_common_generation_per_group['version'] == row['version'])
        ]
        if not match.empty:
            return match.iloc[0]['generation']
    return row['generation']

combustion_cars['generation'] = combustion_cars.apply(fill_generation, axis=1)

combustion_cars.generation.isna().sum()
combustion_cars['version'] = combustion_cars.apply(fill_version, axis=1)

missing_gen_or_ver = combustion_cars[
    combustion_cars['version'].isna() | combustion_cars['generation'].isna()
][['model', 'year']].drop_duplicates().reset_index(drop=True)

combustion_cars = combustion_cars.dropna(subset=['version', 'generation']).reset_index(drop=True)

combustion_cars.gearbox.fillna('automatic')

combustion_cars.engine_capacity.isna().sum() #8

np.int64(8)

In [274]:
rows_with_nan_capacity = (
    combustion_cars[combustion_cars['engine_capacity'].isna()]
    [['model', 'year', 'generation', 'version']]
    .drop_duplicates()
    .to_dict(orient='records')
)

most_common_engine_capacity = (
    combustion_cars.dropna(subset=['engine_capacity'])
    .groupby(['model', 'year', 'version', 'generation'])['engine_capacity']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
    .rename(columns={'engine_capacity': 'most_common_capacity'})
)

def fill_engine_capacity(row):
    if pd.isna(row['engine_capacity']):
        match = most_common_engine_capacity[
            (most_common_engine_capacity['model'] == row['model']) &
            (most_common_engine_capacity['year'] == row['year']) &
            (most_common_engine_capacity['version'] == row['version']) &
            (most_common_engine_capacity['generation'] == row['generation'])
        ]
        if not match.empty:
            return match.iloc[0]['most_common_capacity']
    return row['engine_capacity']

combustion_cars['engine_capacity'] = combustion_cars.apply(fill_engine_capacity, axis=1)

combustion_cars


,is_imported_car,model,year,mileage,engine_capacity,fuel_type,engine_power,gearbox,transmission,bluetooth_interface,...,maker_warranty_valid_until_date,maker_warranty_valid_until_km,convertible_top_type,monthly_payment,remaining_payments,residual_value,down_payment,avg_consumption,show_pir,price_amount
0,1,seria-7,2011,250000,2993.0,diesel,245.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70000.0
1,0,3gt,2016,250147,1995.0,diesel,150.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62900.0
2,0,seria-5,2018,83000,2993.0,diesel,400.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239900.0
3,1,seria-6,2017,133000,2993.0,diesel,313.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139900.0
4,1,seria-5,2021,42000,1995.0,diesel,190.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0,x6,2010,220000,2993.0,diesel,245.0,automatic,all-wheel-auto,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68500.0
10237,1,seria-8,2019,103700,2998.0,petrol,340.0,automatic,rear-wheel,1.0,...,NaN,NaN,textile-top,NaN,NaN,NaN,NaN,NaN,NaN,223900.0
10238,0,x6,2017,92614,2993.0,diesel,313.0,automatic,all-wheel-auto,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.0
10239,1,3gt,2019,199800,1995.0,diesel,143.0,manual,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51900.0


In [275]:
combustion_cars.fuel_type.replace({'petrol': 'benzine'})

mask = combustion_cars.engine_power.isna()
combustion_cars[mask]

# 1. Grupujemy rekordy z niepustym engine_power
most_common_engine_power = (
    combustion_cars.dropna(subset=['engine_power'])
    .groupby(['model', 'version', 'generation', 'engine_capacity', 'fuel_type'])['engine_power']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
    .rename(columns={'engine_power': 'most_common_power'})
)

# 2. Funkcja do uzupełniania brakujących wartości
def fill_engine_power(row):
    if pd.isna(row['engine_power']):
        match = most_common_engine_power[
            (most_common_engine_power['model'] == row['model']) &
            (most_common_engine_power['version'] == row['version']) &
            (most_common_engine_power['generation'] == row['generation']) &
            (most_common_engine_power['engine_capacity'] == row['engine_capacity']) &
            (most_common_engine_power['fuel_type'] == row['fuel_type'])
        ]
        if not match.empty:
            return match.iloc[0]['most_common_power']
    return row['engine_power']

# 3. Zastosowanie funkcji
combustion_cars['engine_power'] = combustion_cars.apply(fill_engine_power, axis=1)

# 4. Sprawdzenie, czy wszystko zostało uzupełnione
print("Brakujące engine_power po uzupełnieniu:", combustion_cars['engine_power'].isna().sum())


Brakujące engine_power po uzupełnieniu: 3


In [276]:
combustion_cars.gearbox = combustion_cars.gearbox.fillna('automatic')

In [277]:
records_to_drop = combustion_cars[combustion_cars['engine_power'].isna()]

combustion_cars = combustion_cars.dropna(subset=['engine_power'])

combustion_cars


,is_imported_car,model,year,mileage,engine_capacity,fuel_type,engine_power,gearbox,transmission,bluetooth_interface,...,maker_warranty_valid_until_date,maker_warranty_valid_until_km,convertible_top_type,monthly_payment,remaining_payments,residual_value,down_payment,avg_consumption,show_pir,price_amount
0,1,seria-7,2011,250000,2993.0,diesel,245.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70000.0
1,0,3gt,2016,250147,1995.0,diesel,150.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62900.0
2,0,seria-5,2018,83000,2993.0,diesel,400.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239900.0
3,1,seria-6,2017,133000,2993.0,diesel,313.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139900.0
4,1,seria-5,2021,42000,1995.0,diesel,190.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0,x6,2010,220000,2993.0,diesel,245.0,automatic,all-wheel-auto,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68500.0
10237,1,seria-8,2019,103700,2998.0,petrol,340.0,automatic,rear-wheel,1.0,...,NaN,NaN,textile-top,NaN,NaN,NaN,NaN,NaN,NaN,223900.0
10238,0,x6,2017,92614,2993.0,diesel,313.0,automatic,all-wheel-auto,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.0
10239,1,3gt,2019,199800,1995.0,diesel,143.0,manual,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51900.0


In [278]:
combustion_cars = combustion_cars.copy()

# Filling transmission col
combustion_cars.transmission.isna().sum()

group_cols = ['model', 'version', 'generation', 'gearbox', 'fuel_type', 'engine_capacity', 'engine_power']

# Grupowanie i wybór najczęstszej wartości transmission
most_common_transmission = (
    combustion_cars.dropna(subset=['transmission'])  # tylko tam, gdzie transmission jest znane
    .groupby(group_cols)['transmission']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
    .rename(columns={'transmission': 'most_common_transmission'})
)
def fill_transmission(row):
    if pd.isna(row['transmission']):
        match = most_common_transmission[
            (most_common_transmission['model'] == row['model']) &
            (most_common_transmission['version'] == row['version']) &
            (most_common_transmission['generation'] == row['generation']) &
            (most_common_transmission['gearbox'] == row['gearbox']) &
            (most_common_transmission['fuel_type'] == row['fuel_type']) &
            (most_common_transmission['engine_capacity'] == row['engine_capacity']) &
            (most_common_transmission['engine_power'] == row['engine_power'])
        ]
        if not match.empty:
            return match.iloc[0]['most_common_transmission']
    return row['transmission']

combustion_cars['transmission'] = combustion_cars.apply(fill_transmission, axis=1)

print("Pozostałe braki w kolumnie transmission:", combustion_cars['transmission'].isna().sum()) # z 550 na 148

# hierarchiczne podejscie przez zmniejszanie cech #2
still_missing = combustion_cars[combustion_cars['transmission'].isna()].copy()


group_cols = ['model', 'version', 'gearbox', 'fuel_type', 'engine_capacity', 'engine_power']

most_common_transmission_2 = (
    combustion_cars.dropna(subset=['transmission'])
    .groupby(group_cols)['transmission']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
    .rename(columns={'transmission': 'most_common_transmission'})
)

def fill_transmission_v2(row):
    if pd.isna(row['transmission']):
        match = most_common_transmission_2[
            (most_common_transmission_2['model'] == row['model']) &
            (most_common_transmission_2['version'] == row['version']) &
            (most_common_transmission_2['gearbox'] == row['gearbox']) &
            (most_common_transmission_2['fuel_type'] == row['fuel_type']) &
            (most_common_transmission_2['engine_capacity'] == row['engine_capacity']) &
            (most_common_transmission_2['engine_power'] == row['engine_power'])
        ]
        if not match.empty:
            return match.iloc[0]['most_common_transmission']
    return row['transmission']

combustion_cars['transmission'] = combustion_cars.apply(fill_transmission_v2, axis=1)
print("Pozostałe braki po drugim podejściu:", combustion_cars['transmission'].isna().sum()) # z 148 na 140

#2 kontynuuje

# Krok 3 – mniej cech
group_cols_3 = ['model', 'gearbox', 'fuel_type']

most_common_transmission_3 = (
    combustion_cars.dropna(subset=['transmission'])
    .groupby(group_cols_3)['transmission']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
    .rename(columns={'transmission': 'most_common_transmission'})
)

def fill_transmission_v3(row):
    if pd.isna(row['transmission']):
        match = most_common_transmission_3[
            (most_common_transmission_3['model'] == row['model']) &
            (most_common_transmission_3['gearbox'] == row['gearbox']) &
            (most_common_transmission_3['fuel_type'] == row['fuel_type'])
        ]
        if not match.empty:
            return match.iloc[0]['most_common_transmission']
    return row['transmission']

combustion_cars['transmission'] = combustion_cars.apply(fill_transmission_v3, axis=1)
print("Pozostałe braki po kroku 3:", combustion_cars['transmission'].isna().sum())


Pozostałe braki w kolumnie transmission: 148
Pozostałe braki po drugim podejściu: 140
Pozostałe braki po kroku 3: 0


In [279]:
combustion_cars.damaged.astype('Int32')

combustion_cars.damaged.isna().sum()


# jesli nie podano czy uszkoszkodzony porownujemy cene samochodu uczeniem na podstawie ceny i innych cech
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


labeled = combustion_cars[combustion_cars['damaged'].notna()]
unlabeled = combustion_cars[combustion_cars['damaged'].isna()]

features = [
    'model', 'version', 'generation', 'fuel_type', 'gearbox',
    'transmission', 'body_type', 'mileage', 'engine_capacity', 'engine_power', 'price_amount'
]

X = labeled[features]
y = labeled['damaged'].astype(int)

X_unlabeled = unlabeled[features]
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int', 'float']).columns.tolist()

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# 3. Model i pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# 4. Trening
model.fit(X, y)

# 5. Predykcja dla brakujących
y_pred = model.predict(X_unlabeled)

# 6. Wstawiamy do oryginalnej ramki danych
combustion_cars.loc[unlabeled.index, 'damaged'] = y_pred

# 7. Sprawdzenie efektu
print("Pozostałe NaNy w damaged:", combustion_cars['damaged'].isna().sum())


Pozostałe NaNy w damaged: 0


In [280]:
combustion_cars.door_count.isna().sum() # 40

# 1. Grupujemy po logicznych cechach i znajdujemy najczęstszy door_count
most_common_door_count = (
    combustion_cars.dropna(subset=['door_count'])
    .groupby(['model', 'generation', 'version', 'body_type', 'nr_seats'])['door_count']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
    .rename(columns={'door_count': 'most_common_door_count'})
)

# 2. Funkcja do przypisywania najczęstszej wartości
def fill_door_count(row):
    if pd.isna(row['door_count']):
        match = most_common_door_count[
            (most_common_door_count['model'] == row['model']) &
            (most_common_door_count['generation'] == row['generation']) &
            (most_common_door_count['version'] == row['version']) &
            (most_common_door_count['body_type'] == row['body_type']) & 
            (most_common_door_count['nr_seats'] == row['nr_seats'])
        ]
        if not match.empty:
            return match.iloc[0]['most_common_door_count']
    return row['door_count']

# 3. Zastosowanie
combustion_cars['door_count'] = combustion_cars.apply(fill_door_count, axis=1)

# 4. Sprawdzenie efektu
print("Pozostałe braki w door_count:", combustion_cars['door_count'].isna().sum())

mask = combustion_cars['door_count'].isna()
combustion_cars[mask] 

# reszta manualnie z internetu bo tylko 3 rekordy do uzupelnienia
most_common_door_count_x1 = combustion_cars[(combustion_cars.body_type == 'compact') & (combustion_cars.model == 'x1')]['door_count'].value_counts().idxmax()
most_common_door_seria_3 = combustion_cars[(combustion_cars.body_type == 'compact') & (combustion_cars.model == 'seria-3')]['door_count'].value_counts().idxmax()

mask1 = (combustion_cars['door_count'].isna()) & (combustion_cars.model == 'x1')
mask2 = ((combustion_cars['door_count'].isna()) & (combustion_cars.model == 'seria-3'))

combustion_cars.loc[mask1, 'door_count'] = most_common_door_count_x1
combustion_cars.loc[mask2, 'door_count'] = most_common_door_seria_3

combustion_cars

Pozostałe braki w door_count: 10


,is_imported_car,model,year,mileage,engine_capacity,fuel_type,engine_power,gearbox,transmission,bluetooth_interface,...,maker_warranty_valid_until_date,maker_warranty_valid_until_km,convertible_top_type,monthly_payment,remaining_payments,residual_value,down_payment,avg_consumption,show_pir,price_amount
0,1,seria-7,2011,250000,2993.0,diesel,245.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70000.0
1,0,3gt,2016,250147,1995.0,diesel,150.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62900.0
2,0,seria-5,2018,83000,2993.0,diesel,400.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239900.0
3,1,seria-6,2017,133000,2993.0,diesel,313.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139900.0
4,1,seria-5,2021,42000,1995.0,diesel,190.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0,x6,2010,220000,2993.0,diesel,245.0,automatic,all-wheel-auto,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68500.0
10237,1,seria-8,2019,103700,2998.0,petrol,340.0,automatic,rear-wheel,1.0,...,NaN,NaN,textile-top,NaN,NaN,NaN,NaN,NaN,NaN,223900.0
10238,0,x6,2017,92614,2993.0,diesel,313.0,automatic,all-wheel-auto,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.0
10239,1,3gt,2019,199800,1995.0,diesel,143.0,manual,rear-wheel,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51900.0


In [281]:
#nr_seats
combustion_cars.nr_seats.isna().sum() # 836

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

features = ['model', 'version', 'generation', 'body_type', 'door_count']
target = 'nr_seats'

df_full = combustion_cars.dropna(subset=features + [target])
df_missing = combustion_cars[combustion_cars[target].isna()]

# 1. Encoding
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_encoded = encoder.fit_transform(df_full[features])
y = df_full[target].astype(int)

# 2. Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# 3. Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 4. Accuracy
y_pred = model.predict(X_test)
print("Dokładność modelu:", accuracy_score(y_test, y_pred))

# 5. Predict missing
X_missing = encoder.transform(df_missing[features])
y_missing_pred = model.predict(X_missing)

# 6. Fill back
combustion_cars.loc[df_missing.index, 'nr_seats'] = y_missing_pred

# 7. Sprawdzenie braków
print("Pozostałe braki w 'nr_seats':", combustion_cars['nr_seats'].isna().sum())




Dokładność modelu: 0.9580010632642212
Pozostałe braki w 'nr_seats': 0


In [282]:
# usuwanie kolumn gdzie ponad 50% wartosci to NaN

threshold = 0.5
nan_ratio = combustion_cars.isna().mean()

cols_to_drop = nan_ratio[nan_ratio > threshold].index
combustion_cars.drop(columns=cols_to_drop, inplace=True)

print(f"Usunięto {len(cols_to_drop)} kolumn z więcej niż 50% brakami.")
print("Pozostałe kolumny:", combustion_cars.columns.tolist())


Usunięto 136 kolumn z więcej niż 50% brakami.
Pozostałe kolumny: ['is_imported_car', 'model', 'year', 'mileage', 'engine_capacity', 'fuel_type', 'engine_power', 'gearbox', 'transmission', 'bluetooth_interface', 'damaged', 'body_type', 'door_count', 'nr_seats', 'color', 'colour_type', 'radio', 'rhd', 'hands_free_system', 'usb_in', 'registered', 'original_owner', 'no_accident', 'service_record', 'navigation_system', 'soundsystem', 'air_conditioning_type', 'upholstery_type', 'heated_seat_driver', 'heated_seat_passenger', 'power_windows_front', 'cruisecontrol_type', 'headlight_lamp_type', 'park_distance_control_rear', 'door_mirror_electrically_adjustable_in_general', 'door_mirrors_heated', 'traction_control', 'fog_lamps', 'startstop_system', 'power_steering', 'antilock_brake_system', 'esp', 'passenger_airbag', 'side_airbag_driver_and_passenger', 'child_seat_fixation', 'new_used', 'version', 'generation', 'urban_consumption', 'power_windows_rear', 'version_label', 'country_origin', 'leather

In [283]:
# color

combustion_cars.color.isna().sum() # 0

np.int64(0)

In [284]:
# colour_type

combustion_cars.colour_type.isna().sum() # 4437

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Kolumny wejściowe
features = ['model', 'version', 'generation', 'year', 'color']
target = 'colour_type'

# Drop wiersze, gdzie brakuje kolumn wejściowych lub celu
df_train = combustion_cars.dropna(subset=features + [target]).copy()
df_missing = combustion_cars[combustion_cars[target].isna()].copy()

# Label encoding
label_encoders = {}
for col in features + [target]:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col].astype(str))
    label_encoders[col] = le

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(
    df_train[features], df_train[target], test_size=0.2, random_state=42)

# Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Ewaluacja
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy na znanych danych (validation):", round(accuracy * 100, 2), "%")

# Przekształć brakujące dane tymi samymi encoderami
for col in features:
    le = label_encoders[col]
    df_missing[col] = df_missing[col].astype(str)
    df_missing[col] = df_missing[col].apply(lambda x: x if x in le.classes_ else 'unknown')
    if 'unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'unknown')
    df_missing[col] = le.transform(df_missing[col])

# Przewidywanie brakujących wartości
predicted_colour_type = model.predict(df_missing[features])
df_missing[target] = label_encoders[target].inverse_transform(predicted_colour_type)

# Aktualizacja oryginalnego DataFrame
combustion_cars.loc[df_missing.index, target] = df_missing[target]




Accuracy na znanych danych (validation): 84.75 %


In [285]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Uzupełniamy: jeśli damaged = 1, to no_accident = 0
combustion_cars.loc[combustion_cars['damaged'] == 1, 'no_accident'] = 0

# 2. Wybieramy dane do uczenia
features = ['model', 'version', 'generation', 'year', 'body_type', 'price_amount']
target = 'no_accident'

df_train = combustion_cars.dropna(subset=features + [target]).copy()
df_missing = combustion_cars[combustion_cars[target].isna()].copy()

# 3. Label encoding dla kolumn tekstowych
label_encoders = {}
for col in features:
    if df_train[col].dtype == 'object':
        le = LabelEncoder()
        df_train[col] = le.fit_transform(df_train[col].astype(str))
        label_encoders[col] = le

# 4. Split danych do walidacji
X = df_train[features]
y = df_train[target].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Trenujemy model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 6. Ewaluacja dokładności
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Dokładność predykcji no_accident:", round(accuracy * 100, 2), "%")

# 7. Przygotowanie danych brakujących (kodowanie)
for col in features:
    if col in label_encoders:
        le = label_encoders[col]
        df_missing[col] = df_missing[col].astype(str)
        # Obsługa wcześniej niewidzianych wartości
        df_missing[col] = df_missing[col].apply(lambda x: x if x in le.classes_ else 'unknown')
        if 'unknown' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'unknown')
        df_missing[col] = le.transform(df_missing[col])

# 8. Predykcja brakujących wartości
df_missing[target] = model.predict(df_missing[features])

# 9. Zapisz z powrotem do oryginalnego dataframe
combustion_cars.loc[df_missing.index, target] = df_missing[target]

# 10. Podsumowanie
print("Brakujące wartości no_accident po uzupełnieniu:", combustion_cars['no_accident'].isna().sum())


Dokładność predykcji no_accident: 85.49 %
Brakujące wartości no_accident po uzupełnieniu: 0


In [286]:
combustion_cars.loc[:, 'bluetooth_interface'] = (
    combustion_cars['bluetooth_interface'].fillna(0).astype('int32')
)

combustion_cars.bluetooth_interface

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
10236    1.0
10237    1.0
10238    1.0
10239    1.0
10240    1.0
Name: bluetooth_interface, Length: 10238, dtype: float64

In [287]:
# Obliczamy procent braków w każdej kolumnie
missing_percent = combustion_cars.isna().mean() * 100

# Filtrujemy kolumny, gdzie ponad 40% to NaN
high_missing_columns = missing_percent[missing_percent > 40]



cols_to_drop = ['battery_ownership_model', 'autonomy', 'battery_capacity', 'avg_consumption', 'show_pir']
existing_cols = [col for col in cols_to_drop if col in combustion_cars.columns]

combustion_cars = combustion_cars.drop(columns=existing_cols)



In [288]:
missing_percent = combustion_cars.isna().mean() * 100

# Filtrujemy kolumny, gdzie ponad 40% to NaN
high_missing_columns = missing_percent[missing_percent > 40]

combustion_cars.loc[:, high_missing_columns.index]




,rhd,original_owner,service_record,navigation_system,soundsystem,heated_seat_driver,heated_seat_passenger,fog_lamps,urban_consumption,park_distance_control_front,head_airbags_rear,sports_steering_wheel,follow_me_home
0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN
1,0.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,5.0,NaN,NaN,NaN,NaN
2,0.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,6.0,1.0,1.0,NaN,NaN
3,0.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,6.0,NaN,NaN,NaN,NaN
4,0.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,4.0,1.0,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,8.0,NaN,NaN,NaN,NaN
10237,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0
10238,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0
10239,0.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,5.6,1.0,1.0,1.0,NaN


In [289]:
combustion_cars.park_distance_control_front = combustion_cars.park_distance_control_front.fillna(0).astype('int32')
combustion_cars.head_airbags_rear = combustion_cars.head_airbags_rear.fillna(0).astype('int32')
combustion_cars.sports_steering_wheel = combustion_cars.sports_steering_wheel.fillna(0).astype('int32')
combustion_cars.navigation_system = combustion_cars.navigation_system.fillna(0).astype('int32')
combustion_cars.radio = combustion_cars.radio.fillna(0).astype('int32')
combustion_cars.soundsystem = combustion_cars.soundsystem.fillna(0).astype('int32')

In [290]:
to_drop = ['vat_discount', 'financial_option', 'leasing_concession', 'original_owner', 
           'service_record', 'approval_for_goods', 'combined_consumption', 'extra_urban_consumption',
           'vendors_warranty_valid_until_date', 'ventilated_front_seat', 'tinted_rear_windows', 
           'hill_holder', 'adaptive_light', 'adjustable_suspension', 'power_assisted_brakes', 'active_lane_control_assistant',
           'pre_crash_system', 'autorenew', 'independent_vehicle_heater', 'deactivation_reason_id', 'towbar', 
           'down_payment', 'residual_value', 'remaining_payments', 'monthly_payment', 'convertible_top_type', 'maker_warranty_valid_until_km',
           'maker_warranty_valid_until_date', 'hydro_pneumatic_suspension', 'pre_crash_sound_system', 'rear_seat_with_massage',
           'ventilated_rear_seat', 'rear_transversal_curtain_airbag', 'air_suspension', 'selectable_central_differential_characteristics',
           'roll_over_protection_system', 'seat_belt_airbag_rear', 'acoustic_vehicle_alerting_system', 'vehicle_charging_cable', 
           'quick_charging_function', 'top_electrically_operated', 'windscreenwiper_other', 'central_airbag_driver_and_passenger',
           'active_driver_conditioning_monitoring', 'sunblind_type', 'automatic_emergency_call', 'comfort_suspension', 
           'electronic_controlled_suspension', 'limited_slip_differential_in_general', 'leather_gearshifterswitch',
           'video', 'pause_recommendation_warning', 'rear_cross_traffic_alert', 'side_pre_crash_system', 'rear_pre_crash_system', 
           'active_emergency_brake_assist', 'pedestrian_emergency_brake_assist', 'city_emergency_brake_assist',
           'distribution_of_braking_force_electronically', 'ceramic_composite_brakes', 'traffic_jam_assist', 
           'headlamp_wash', 'basic_autonomous_driving', 'intersection_assist', 'active_speed_sign_recognition', 
           'hill_descent_control', 'curve_trace_assistant', 'active_lane_change_assistant', '360_view_camera', 
           'keyless_go', 'keyless_entry', 'steering_wheel_electrically_adjustable', 'rear_seat_heating', 'memory_seat', 
           'front_seat_with_massage', 'lumbar_adjust_passenger_electric', 'lumbar_adjust_driver_electric', 'sunroof',
           'internet_access', 'sport_suspension', 'runflat_tyres', 'tyre_type', 'follow_me_home', 'automated_high_beam_assist',
           'automatic_heating_control', 'keyless_engine_start', 'rhd','hands_free_system','usb_in', 'registered', 'tuning', 'vat',
            'country_origin', 'service_record_filled', 'upholstery_type', 'alloy_wheels_type', 'automatic_dimlight_activation',
            'armrest_rear',  'armrest_front'           ]


existing_cols = [col for col in to_drop if col in combustion_cars.columns]

combustion_cars = combustion_cars.drop(columns=existing_cols)

len(combustion_cars.columns.tolist())


54

In [291]:
combustion_cars

,is_imported_car,model,year,mileage,engine_capacity,fuel_type,engine_power,gearbox,transmission,bluetooth_interface,...,leather_steering_wheel,multi_functional_steering_wheel,rain_sensor,park_distance_control_front,tyre_pressure_control,driver_airbag,head_airbags_front,head_airbags_rear,sports_steering_wheel,price_amount
0,1,seria-7,2011,250000,2993.0,diesel,245.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,70000.0
1,0,3gt,2016,250147,1995.0,diesel,150.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,62900.0
2,0,seria-5,2018,83000,2993.0,diesel,400.0,automatic,all-wheel-permanent,1.0,...,1.0,1.0,1.0,1,1.0,1.0,1.0,1,0,239900.0
3,1,seria-6,2017,133000,2993.0,diesel,313.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,139900.0
4,1,seria-5,2021,42000,1995.0,diesel,190.0,automatic,rear-wheel,1.0,...,1.0,1.0,1.0,1,1.0,NaN,NaN,0,1,192900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0,x6,2010,220000,2993.0,diesel,245.0,automatic,all-wheel-auto,1.0,...,1.0,1.0,1.0,0,1.0,1.0,1.0,0,0,68500.0
10237,1,seria-8,2019,103700,2998.0,petrol,340.0,automatic,rear-wheel,1.0,...,1.0,NaN,1.0,1,NaN,1.0,1.0,1,1,223900.0
10238,0,x6,2017,92614,2993.0,diesel,313.0,automatic,all-wheel-auto,1.0,...,1.0,1.0,1.0,1,1.0,1.0,1.0,0,1,184500.0
10239,1,3gt,2019,199800,1995.0,diesel,143.0,manual,rear-wheel,1.0,...,1.0,1.0,1.0,1,NaN,1.0,1.0,1,1,51900.0


In [292]:
#

combustion_cars.air_conditioning_type.value_counts(dropna=False)

air_conditioning_type
dualzone-automatic-climate-control          3875
automatic-climate-control                   2591
NaN                                         1195
trizone-automatic-climate-control           1177
4-or-more-zone-automatic-climate-control     702
air-conditioning                             698
Name: count, dtype: int64

In [293]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

# Kopia DataFrame
df = combustion_cars.copy()

# Kolumny do użycia
cat_features = ['model', 'version', 'generation', 'body_type', 'gearbox', 'transmission', 'fuel_type']
numeric_features = ['year']
target_col = 'air_conditioning_type'

# Wiersze, gdzie air_conditioning_type jest znane
df_known = df.dropna(subset=[target_col]).copy()

# Kodowanie zmiennych kategorycznych
encoders = {}
for col in cat_features + [target_col]:
    df_known[col] = df_known[col].astype(str)
    le = LabelEncoder()
    df_known[col] = le.fit_transform(df_known[col])
    encoders[col] = le

# Cecha X i y
X = df_known[cat_features + numeric_features]
y = df_known[target_col]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Trening modelu
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Accuracy
accuracy = clf.score(X_test, y_test)
print("Accuracy (dokładność) na znanych danych:", accuracy)

# --------------------
# Wypełnianie braków
# --------------------
df_missing = df[df[target_col].isna()].copy()

# Kodowanie cech kategorycznych
for col in cat_features:
    df_missing[col] = df_missing[col].astype(str)
    df_missing[col] = df_missing[col].apply(
        lambda val: val if val in encoders[col].classes_ else 'UNK'
    )
    if 'UNK' not in encoders[col].classes_:
        encoders[col].classes_ = np.append(encoders[col].classes_, 'UNK')
    df_missing[col] = encoders[col].transform(df_missing[col])

# Przewidywanie braków
X_missing = df_missing[cat_features + numeric_features]
df_missing[target_col] = clf.predict(X_missing)

# Odkodowanie wartości
df_known[target_col] = encoders[target_col].inverse_transform(df_known[target_col])
df_missing[target_col] = encoders[target_col].inverse_transform(df_missing[target_col])

# Łączenie
df_filled = pd.concat([df_known, df_missing]).sort_index()

# Aktualizacja combustion_cars
combustion_cars['air_conditioning_type'] = df_filled['air_conditioning_type']

# Sprawdzenie braków
print("Liczba brakujących wartości w air_conditioning_type:",
      combustion_cars['air_conditioning_type'].isna().sum())


Accuracy (dokładność) na znanych danych: 0.6323935876174682
Liczba brakujących wartości w air_conditioning_type: 0


In [294]:
combustion_cars

,is_imported_car,model,year,mileage,engine_capacity,fuel_type,engine_power,gearbox,transmission,bluetooth_interface,...,leather_steering_wheel,multi_functional_steering_wheel,rain_sensor,park_distance_control_front,tyre_pressure_control,driver_airbag,head_airbags_front,head_airbags_rear,sports_steering_wheel,price_amount
0,1,seria-7,2011,250000,2993.0,diesel,245.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,70000.0
1,0,3gt,2016,250147,1995.0,diesel,150.0,automatic,rear-wheel,1.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,62900.0
2,0,seria-5,2018,83000,2993.0,diesel,400.0,automatic,all-wheel-permanent,1.0,...,1.0,1.0,1.0,1,1.0,1.0,1.0,1,0,239900.0
3,1,seria-6,2017,133000,2993.0,diesel,313.0,automatic,all-wheel-permanent,1.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,139900.0
4,1,seria-5,2021,42000,1995.0,diesel,190.0,automatic,rear-wheel,1.0,...,1.0,1.0,1.0,1,1.0,NaN,NaN,0,1,192900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0,x6,2010,220000,2993.0,diesel,245.0,automatic,all-wheel-auto,1.0,...,1.0,1.0,1.0,0,1.0,1.0,1.0,0,0,68500.0
10237,1,seria-8,2019,103700,2998.0,petrol,340.0,automatic,rear-wheel,1.0,...,1.0,NaN,1.0,1,NaN,1.0,1.0,1,1,223900.0
10238,0,x6,2017,92614,2993.0,diesel,313.0,automatic,all-wheel-auto,1.0,...,1.0,1.0,1.0,1,1.0,1.0,1.0,0,1,184500.0
10239,1,3gt,2019,199800,1995.0,diesel,143.0,manual,rear-wheel,1.0,...,1.0,1.0,1.0,1,NaN,1.0,1.0,1,1,51900.0


In [295]:


combustion_cars.heated_seat_passenger.fillna(0).astype('int32')
combustion_cars.heated_seat_passenger.fillna(0).astype('int32')
combustion_cars.power_windows_front.fillna(0).astype('int32')
combustion_cars.park_distance_control_rear.fillna(0).astype('int32')
combustion_cars.traction_control.fillna(0).astype('int32')
combustion_cars.fog_lamps.fillna(0).astype('int32')
combustion_cars.power_steering.fillna(0).astype('int32')
combustion_cars.esp.fillna(0).astype('int32')
combustion_cars.passenger_airbag.fillna(0).astype('int32')
combustion_cars.side_airbag_driver_and_passenger.fillna(0).astype('int32')

to_drop = ['cruisecontrol_type', 'headlight_lamp_type', 'door_mirror_electrically_adjustable_in_general', 'door_mirrors_heated', 'startstop_system',
           'antilock_brake_system', 'child_seat_fixation', 'power_windows_rear', 'leather_steering_wheel', 'multi_functional_steering_wheel',
           'rain_sensor', 'tyre_pressure_control', 'driver_airbag', 'head_airbags_front']


existing_cols = [col for col in to_drop if col in combustion_cars.columns]

combustion_cars = combustion_cars.drop(columns=existing_cols)

combustion_cars

,is_imported_car,model,year,mileage,engine_capacity,fuel_type,engine_power,gearbox,transmission,bluetooth_interface,...,side_airbag_driver_and_passenger,new_used,version,generation,urban_consumption,version_label,park_distance_control_front,head_airbags_rear,sports_steering_wheel,price_amount
0,1,seria-7,2011,250000,2993.0,diesel,245.0,automatic,rear-wheel,1.0,...,1.0,0,ver-730d,gen-f01-2008-2015,9.0,730d,0,0,0,70000.0
1,0,3gt,2016,250147,1995.0,diesel,150.0,automatic,rear-wheel,1.0,...,1.0,0,ver-318d-luxury-line,gen-f34-2013-2021,5.0,318d Luxury Line,0,0,0,62900.0
2,0,seria-5,2018,83000,2993.0,diesel,400.0,automatic,all-wheel-permanent,1.0,...,1.0,0,ver-m550d-xdrive,gen-g30-2017,6.0,M550d xDrive,1,1,0,239900.0
3,1,seria-6,2017,133000,2993.0,diesel,313.0,automatic,all-wheel-permanent,1.0,...,1.0,0,ver-640d-xdrive-edycja-m-sport,gen-f12-13-2011,6.0,640d xDrive Edycja M Sport,0,0,0,139900.0
4,1,seria-5,2021,42000,1995.0,diesel,190.0,automatic,rear-wheel,1.0,...,NaN,0,ver-520d-m-sport,gen-g30-2017,4.0,520d M Sport,1,0,1,192900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0,x6,2010,220000,2993.0,diesel,245.0,automatic,all-wheel-auto,1.0,...,1.0,0,ver-30d-xdrive,gen-e71-2008-2014,8.0,30d xDrive,0,0,0,68500.0
10237,1,seria-8,2019,103700,2998.0,petrol,340.0,automatic,rear-wheel,1.0,...,1.0,0,ver-840i,gen-g15-2018,NaN,840i,1,1,1,223900.0
10238,0,x6,2017,92614,2993.0,diesel,313.0,automatic,all-wheel-auto,1.0,...,1.0,0,ver-xdrive40d,gen-f16-2014,NaN,xDrive40d,1,0,1,184500.0
10239,1,3gt,2019,199800,1995.0,diesel,143.0,manual,rear-wheel,1.0,...,1.0,0,ver-318d-gt-luxury-line,gen-f34-2013-2021,5.6,318d GT Luxury Line,1,1,1,51900.0


In [296]:
combustion_cars.isna().sum() # 0

is_imported_car                        0
model                                  0
year                                   0
mileage                                0
engine_capacity                        0
fuel_type                              0
engine_power                           0
gearbox                                0
transmission                           0
bluetooth_interface                    0
damaged                                0
body_type                              0
door_count                             7
nr_seats                               0
color                                  0
colour_type                            0
radio                                  0
no_accident                            0
navigation_system                      0
soundsystem                            0
air_conditioning_type                  0
heated_seat_driver                  4613
heated_seat_passenger               4743
power_windows_front                 2427
park_distance_co

In [297]:
combustion_cars = combustion_cars.drop(columns=['version_label'])

In [298]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

# Kopia danych
df = combustion_cars.copy()

# Lista cech do modelu
features = [
    'model',
    'year',
    'engine_capacity',
    'engine_power',
    'fuel_type',
    'gearbox',
    'transmission',
    'body_type'
]
target_col = 'urban_consumption'

# Wiersze z dostępnym targetem
df_known = df.dropna(subset=[target_col]).copy()

# Upewniamy się, że dane kategoryczne są typu string
cat_features = ['model', 'fuel_type', 'gearbox', 'transmission', 'body_type']
for col in cat_features:
    df_known[col] = df_known[col].astype(str)

# Label Encoding
encoders = {}
for col in cat_features:
    le = LabelEncoder()
    df_known[col] = le.fit_transform(df_known[col])
    encoders[col] = le

# Cecha X i y
X = df_known[features]
y = df_known[target_col]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Trening modelu regresyjnego
reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train)

# Predykcja i ocena
y_pred = reg.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R² score (jakość modelu regresji): {r2:.4f}")


R² score (jakość modelu regresji): 0.8676


In [299]:
# Wybór wierszy z brakującą wartością
df_missing = df[df[target_col].isna()].copy()

# Upewniamy się, że kolumny kategoryczne są typu string
for col in cat_features:
    df_missing[col] = df_missing[col].astype(str)

    # Jeśli pojawiły się nowe kategorie, zamień na 'UNK' i dodaj do LabelEncoder
    df_missing[col] = df_missing[col].apply(
        lambda val: val if val in encoders[col].classes_ else 'UNK'
    )
    if 'UNK' not in encoders[col].classes_:
        encoders[col].classes_ = np.append(encoders[col].classes_, 'UNK')
    
    df_missing[col] = encoders[col].transform(df_missing[col])

# Tworzenie X do predykcji
X_missing = df_missing[features]

# Predykcja brakujących wartości
df_missing[target_col] = reg.predict(X_missing)

# ✅ Zaokrąglenie do 1 miejsca po przecinku
df_missing[target_col] = df_missing[target_col].round(1)

# Uzupełnienie braków w oryginalnym DataFrame
df.update(df_missing[[target_col]])

# Potwierdzenie
print("Liczba brakujących wartości po uzupełnieniu:",
      df[target_col].isna().sum())


Liczba brakujących wartości po uzupełnieniu: 0


In [300]:
df.isna().sum() # 0

is_imported_car                        0
model                                  0
year                                   0
mileage                                0
engine_capacity                        0
fuel_type                              0
engine_power                           0
gearbox                                0
transmission                           0
bluetooth_interface                    0
damaged                                0
body_type                              0
door_count                             7
nr_seats                               0
color                                  0
colour_type                            0
radio                                  0
no_accident                            0
navigation_system                      0
soundsystem                            0
air_conditioning_type                  0
heated_seat_driver                  4613
heated_seat_passenger               4743
power_windows_front                 2427
park_distance_co

In [301]:
columns_to_fill_zero = [
    'heated_seat_driver',
    'heated_seat_passenger',
    'power_windows_front',
    'park_distance_control_rear',
    'traction_control',
    'fog_lamps',
    'power_steering',
    'esp',
    'passenger_airbag',
    'side_airbag_driver_and_passenger'
]

for col in columns_to_fill_zero:
    df[col] = df[col].fillna(0)


In [302]:
df.isna().sum() # 0

is_imported_car                     0
model                               0
year                                0
mileage                             0
engine_capacity                     0
fuel_type                           0
engine_power                        0
gearbox                             0
transmission                        0
bluetooth_interface                 0
damaged                             0
body_type                           0
door_count                          7
nr_seats                            0
color                               0
colour_type                         0
radio                               0
no_accident                         0
navigation_system                   0
soundsystem                         0
air_conditioning_type               0
heated_seat_driver                  0
heated_seat_passenger               0
power_windows_front                 0
park_distance_control_rear          0
traction_control                    0
fog_lamps   

In [303]:
df = df.dropna(subset=['door_count'])
df.to_csv('combustion_cars_complete.csv', index=False)

df


,is_imported_car,model,year,mileage,engine_capacity,fuel_type,engine_power,gearbox,transmission,bluetooth_interface,...,passenger_airbag,side_airbag_driver_and_passenger,new_used,version,generation,urban_consumption,park_distance_control_front,head_airbags_rear,sports_steering_wheel,price_amount
0,1,seria-7,2011,250000,2993.0,diesel,245.0,automatic,rear-wheel,1.0,...,1.0,1.0,0,ver-730d,gen-f01-2008-2015,9.0,0,0,0,70000.0
1,0,3gt,2016,250147,1995.0,diesel,150.0,automatic,rear-wheel,1.0,...,1.0,1.0,0,ver-318d-luxury-line,gen-f34-2013-2021,5.0,0,0,0,62900.0
2,0,seria-5,2018,83000,2993.0,diesel,400.0,automatic,all-wheel-permanent,1.0,...,1.0,1.0,0,ver-m550d-xdrive,gen-g30-2017,6.0,1,1,0,239900.0
3,1,seria-6,2017,133000,2993.0,diesel,313.0,automatic,all-wheel-permanent,1.0,...,1.0,1.0,0,ver-640d-xdrive-edycja-m-sport,gen-f12-13-2011,6.0,0,0,0,139900.0
4,1,seria-5,2021,42000,1995.0,diesel,190.0,automatic,rear-wheel,1.0,...,1.0,0.0,0,ver-520d-m-sport,gen-g30-2017,4.0,1,0,1,192900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0,x6,2010,220000,2993.0,diesel,245.0,automatic,all-wheel-auto,1.0,...,1.0,1.0,0,ver-30d-xdrive,gen-e71-2008-2014,8.0,0,0,0,68500.0
10237,1,seria-8,2019,103700,2998.0,petrol,340.0,automatic,rear-wheel,1.0,...,1.0,1.0,0,ver-840i,gen-g15-2018,8.1,1,1,1,223900.0
10238,0,x6,2017,92614,2993.0,diesel,313.0,automatic,all-wheel-auto,1.0,...,1.0,1.0,0,ver-xdrive40d,gen-f16-2014,7.2,1,0,1,184500.0
10239,1,3gt,2019,199800,1995.0,diesel,143.0,manual,rear-wheel,1.0,...,1.0,1.0,0,ver-318d-gt-luxury-line,gen-f34-2013-2021,5.6,1,1,1,51900.0
